In [ ]:
import pickle
import numpy

In [ ]:
experiment_name = "2024.01.05-04_Mixtral-7B_4bit_zero-shot"

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
#input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

#top7_train_data_sim_to_each_test_data_filename = base_path + "/input/top7_train_data_sim_to_each_test_data_but_with_different_classes_with_2023.12.11_input_data.pickle"

output_data_path = base_path + "/../Mistral/data/output/" + experiment_name
log_filename = base_path + "/../Mistral/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

,cnt,text_id,sdg_true,result
0,20,1376,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...
1,21,3889,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...
2,22,6576,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...


In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
#results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')
results_processed.head(3)

,cnt,text_id,sdg_true,result,result_short,label
0,20,1376,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 13: Clim...,13
1,21,3889,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 0---\n T...,0
2,22,6576,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 3: Good ...,3


In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,result_short,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 11---\n\...,11
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 0---\n\n...,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,<s> [INST] [INST]\n<<SYS>>\nClassify the follo...,.\n<</SYS>>\n[/INST]\n [/INST] ---SDG 16: Peac...,16


In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.37      0.38      0.37        60
           1       0.78      0.58      0.67        60
           2       0.95      0.62      0.75        60
           3       0.70      0.82      0.75        60
           4       0.82      0.62      0.70        60
           5       0.83      0.80      0.81        60
           6       0.97      0.55      0.70        60
           7       1.00      0.70      0.82        60
           8       0.63      0.40      0.49        60
           9       0.62      0.73      0.67        60
          10       0.69      0.33      0.45        60
          11       0.45      0.90      0.60        60
          12       0.84      0.77      0.80        60
          13       0.88      0.62      0.73        60
          14       0.96      0.87      0.91        60
          15       0.79      0.90      0.84        60
          16       0.67      0.70      0.68        60
          17     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [ ]:
results_processed_merged = final
results_processed_merged = results_processed_merged.rename(columns={'label':'predicted_label_clean'})

In [ ]:
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: "0" if int(x)>16 else x)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.21      0.50      0.29        60
           1       0.78      0.58      0.67        60
           2       0.95      0.62      0.75        60
           3       0.70      0.82      0.75        60
           4       0.82      0.62      0.70        60
           5       0.83      0.80      0.81        60
           6       0.97      0.55      0.70        60
           7       1.00      0.70      0.82        60
           8       0.63      0.40      0.49        60
           9       0.62      0.73      0.67        60
          10       0.69      0.33      0.45        60
          11       0.45      0.90      0.60        60
          12       0.84      0.77      0.80        60
          13       0.88      0.62      0.73        60
          14       0.96      0.87      0.91        60
          15       0.79      0.90      0.84        60
          16       0.67      0.70      0.68        60

    accuracy    

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [ ]:
report_df

,precision,recall,f1-score,support
0,20.547945,50.000000,29.126214,60.000000
1,77.777778,58.333333,66.666667,60.000000
2,94.871795,61.666667,74.747475,60.000000
3,70.000000,81.666667,75.384615,60.000000
4,82.222222,61.666667,70.476190,60.000000
5,82.758621,80.000000,81.355932,60.000000
6,97.058824,55.000000,70.212766,60.000000
7,100.000000,70.000000,82.352941,60.000000
8,63.157895,40.000000,48.979592,60.000000
9,61.971831,73.333333,67.175573,60.000000


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

75.1804032843811 67.05882352941175 67.05882352941175 68.67853328131936


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

75.2 67.1 67.1 68.7
